In [141]:
import urllib.request
from urllib.parse import quote_plus
import time
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
import time
import requests
import pandas as pd
import unicodedata
import numpy as np

In [158]:
# 크롬 드라이버 경로 입력
driver = webdriver.Chrome('C:/Users/LG/Desktop/21winter/YBIGTA/edu_session/0202 크롤링/chromedriver.exe')

In [159]:
driver.get('https://instagram.com/explore')
time.sleep(1)

## 아이디와 패스워드 입력
driver.find_element_by_name('username').send_keys('')
driver.find_element_by_name('password').send_keys('')
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]/button').click()

In [156]:
tag_table = pd.read_excel('해시태그 정리.xlsx')
tag_category = tag_table.columns
tag_table

,일상,육아,여행/장소,취미/예술,패션,음식,애완동물
0,일상,육아,대구,럽스타그램,데일리룩,먹스타그램,멍스타그램
1,데일리,육아스타그램,홍대,사진,오오티디,맛스타그램,고양이
2,셀카,육아소통,강남,사랑해,패션,카페,반려견
3,셀스타그램,딸스타그램,광주,운동,옷스타그램,먹방,강아지
4,셀피,육아맘,해운대,네일아트,NaN,맛집,냥스타그램
5,얼스타그램,맘스타그램,제주도,젤네일,NaN,커피,개스타그램
6,일상스타그램,아들스타그램,서면,인테리어,NaN,카페스타그램,펫스타그램
7,주말,도치맘,대전,운동하는여자,NaN,NaN,댕댕이
8,다이어트,애스타그램,이태원,네일,NaN,NaN,NaN
9,데이트,인스타베이비,제주도,방탄소술단,NaN,NaN,NaN


In [145]:
def scroll(drive, scroll_num):
    # how_many_scroll만큼 맨 밑으로 스크롤하고 맨 위로 돌아오기
    for _ in range(scroll_num):
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
        time.sleep(1)
    driver.execute_script('window.scrollTo(0,0);')

    # 첫 번째 게시물
    driver.find_element_by_class_name('_9AhH0').click()

In [171]:
image_name_list = []
image_url_list = []
contents_list = []
location_list = []
hashtags_list = []

In [146]:
def initialize():
    image_name_list = []
    image_url_list = []
    contents_list = []
    location_list = []
    hashtags_list = []

In [147]:
# 이미지를 따로 보관할 'img'라는 디렉토리를 미리 생성
def crawl(driver, crawl_num, tag):
    n = 0
    for i in range(crawl_num):
        html = driver.page_source
        soup = bs(html, 'lxml')
        time.sleep(1)

        img_url = soup.select('.KL4Bh')[n].img['src']
        image_name = tag + '_' + str(n) + '.jpg'
        with urllib.request.urlopen(img_url) as f:
            with open('./img/' + tag + '_' + str(n) + '.jpg', 'wb') as h:
                img = f.read()
                h.write(img)

        location = driver.find_elements_by_class_name('O4GlU')
        if len(location) == 0:
            location = np.nan
        else:
            location = location[0].text
            location = unicodedata.normalize('NFC', location)

        contents = driver.find_elements_by_xpath('/html/body/div[5]/div[2]/div/article/div[3]/div[1]/ul/div/li/div/div/div[2]/span')
        if len(contents) == 0:
            contents = np.nan
        else:
            contents = contents[0].text.split('#')[0]
            contents = unicodedata.normalize('NFC', contents)

        # 답글 열기 - test
        replies = driver.find_elements_by_class_name('EizgU')
        for reply in replies:
            reply.click()
        time.sleep(1)

        hashtags = ''
        tag_list = driver.find_elements_by_class_name('xil3i')
        if len(tag_list) == 0:
            hashtags = np.nan
        else:    
            for k in range(len(tag_list)):
                hashtags += tag_list[k].text
            hashtags = unicodedata.normalize('NFC', hashtags)

        image_url_list.append(img_url)
        image_name_list.append(image_name)
        location_list.append(location)
        hashtags_list.append(hashtags)
        contents_list.append(contents)

        # 다음 게시물
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(1)
        n += 1
     

In [148]:
def get_url(driver, tag):
    url="https://www.instagram.com/explore/tags/"
    url = url + quote_plus(tag)
    driver.get(url)

    time.sleep(3)

    html = driver.page_source
    soup = bs(html, 'lxml')

검색할 해시태그들을 밑에 tag_list에 넣고 돌리시면 될 것 같아요  
tag_list = tag_table['일상'][3:7]  
tag_list2 = tag_table['패션'][:2]  
  
print(pd.concat([tag_list,tag_list2],ignore_index=True))  
위가 예시이고 이런 식으로 tag_list를 만들어놓고 하시면 될 것 같습니다. Nan값이 들어가게 되면 안되서 인덱스 주의해서 해주세요  
추후에 해시태그들을 추가적으로 더 선정하게 될지를 모르겠어서 이런 방식으로 해봤습니다. 

In [160]:
tag_list = tag_table['일상'][3:7]
tag_list2 = tag_table['패션'][:2]
tag_list = pd.concat([tag_list,tag_list2],ignore_index=True)
tag_list

0     셀스타그램
1        셀피
2     얼스타그램
3    일상스타그램
4      데일리룩
5      오오티디
dtype: object

In [168]:
def get_result():
    dictionary = {'image_name':image_name_list,
                 'image_url':image_url_list,
                 'contents':contents_list,
                 'location':location_list,
                 'hashtags':hashtags_list}
    df = pd.DataFrame(dictionary)
    df.to_csv("data_list.csv", index=True, encoding='utf-8-sig')

메인 프로세스입니다.  
scroll함수의 오른쪽 파라미터가 스크롤 내리는 횟수. 스크롤 내려서 게시물을 많이 확보합니다. 값이 최소 1이어야 합니다..  
crawl의 가운데 crawl_num 파라미터가 게시물 몇 개 크롤링할지.

In [172]:
initialize()
for tag in tag_list:
    get_url(driver, tag)
    time.sleep(1)
    scroll(driver, 10)
    crawl(driver, 5, tag)
get_result()